# Identificação do Cliente de maior valor

# 0.0. Planejamento da Solução ( IOT )

## Input - Entrada

### O problema de negócio

1. Selecionar os mais valiosos clientes para formar o programa de fidelidade **"INSIDERS"**
2. Conjunto de dados com as vendas de produtos durante 1 ano ( Nov 2015 to Dez 2017 )

## Output - Saída

1. A indicação de pessoas para fazer parte do programa de fidelidade **"INSIDERS"**.
2. Relatório com as respostas para as seguintes perguntas:
    - Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - Quantos clientes farão parte do grupo?
    - Quais as principais características desses clientes ?
    - Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - Quais as condições para uma pessoa ser elegível ao Insiders ?
    - Quais as condições para uma pessoa ser removida do Insiders ?
    - Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Quais ações o time de marketing pode realizar para aumentar o faturamento?

## Tasks - Processo

1. **Quem são as pessoas elegíveis para participar do programa de Insiders ?**
    - O que é ser elegível ? O que é um cliente "valioso" para a empresa ?
        - Faturamento:
            - Alto Ticket Médio
            - Alto LTV
            - Baixa Recência ou Alta Frequência ( tempo entre as compras )
            - Alto Basket Size ( quantidade média de produtos comprados )
            - Baixa probabilidade de Churn
            - Previsão alta de LTV
            - Alta propensão de compra

        - Custo:
            - Baixo número de devoluções

        - Experiência:  
            - Média alta de avaliações
            
            
2. **Quantos clientes farão parte do grupo?**
    - Número de clientes
    - % em relação ao total de clients
    
    
3. **Quais as principais características desses clientes ?**
    - Escrever os principais atributos dos clientes
        - Idade
        - País
        - Salário
        
    - Escrever os principais comportamentos de compra dos clients ( métricas de negócio )
        - Vide acima
    
    
4. **Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?**
    - Calcular o faturamento total da empresa durante o ano.
    - Calcular o faturamento (%) apenas do cluster Insiders.
    
    
5. **Qual a expectativa de faturamento desse grupo para os próximos meses ?**
    - Cálculo do LTV do grupo Insiders
    - Séries Temporais ( ARMA, ARIMA, HoltWinter, etc )
    

6. **Quais as condições para uma pessoa ser elegível ao Insiders ?**
    - Qual o período de avaliação ?
    - O "desempenho" do cliente está próximo da média do cluster Insiders. 
    
    
7. **Quais as condições para uma pessoa ser removida do Insiders ?**
    - O "desempenho" do cliente não está mais próximo da média do cluster Insiders. 
    
    
8. **Qual a garantia que o programa Insiders é melhor que o restante da base ?**
    - Teste de Hipóteses
    - Teste A/B
    
    
9. **Quais ações o time de marketing pode realizar para aumentar o faturamento?**
    - Descontos
    - Preferências de escolha
    - Produtos exclusivos

## <font color='red'> Proximos Passos </font>

### Ciclo 01 - Finalizado

1. Feature Engineering
    - Recência
    - Frequência
    - Monetização
 
 
2. Métricas de validação de Clustering
    - WSS ( Within-Cluster Sum of Square )
    - SS ( Silhouette Score )


2. Cluster Analysis
    - Plot 3D
    - Cluster Profile

### Ciclo 02 - Finalizado

1. Feature Engineering
    - Average Ticket


2. Análise de Silhouette
    - Silhouette Analysis  


3. Cluster Visualization
    - UMAP


4. Cluster Análise de Perfil
    - Descrição dos centróides dos cluster

### Ciclo 03 - Finalizado

1. Análise Descritiva
    - Atributos numéricos
    - Atributos categóricos


2. Feature Engineering
    - Average Recency
    - Number of Returns


3. Data preparation
    - Standard Scaler

### Ciclo 04 Finalizado
1. Substituir os dados flatantos
    a. Planejamento de recuperação de 25% dos clientes faltantes.
    b. Execução do plano sobre os dados faltantes.
2. Criação de features diretas.
3. Iniciado com EDA.

### <font color='red'> Para fazer </font>

1. Validar dados NA na criação de novas features
    a. Entender o motivos dos NAs e se está correto desta forma.    
2. Continuar com EDA.

# 0.1. Imports

In [1]:
import numpy   as np
import pandas  as pd
import seaborn as sns
import inflection
import umap.umap_ as umap

from matplotlib import pyplot as plt
from IPython.display import HTML
from sklearn import cluster as c
from sklearn import metrics as m
from sklearn import preprocessing as pp
from plotly import express as px
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

## 0.1. Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Load dataset

In [3]:
# Carregando dados
df_raw = pd.read_csv( '../data/raw/Ecommerce.csv')

In [4]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


# 1.0. Descrição dos Dados

In [5]:
df1 = df_raw.copy()

## 1.1. Renomear Colunas

In [6]:
snakecase = lambda x: inflection.underscore( x )

cols_new = list( map( snakecase, df_raw.columns))

df1.columns = cols_new

df1.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

## 1.2. Dimensões dos dados

In [7]:
print(f'Número de linhas: {df1.shape[0]}')
print(f'Número de Atributos: {df1.shape[1]}')

Número de linhas: 541909
Número de Atributos: 8


## 1.3. Tipo dos Dados

In [8]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [9]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [10]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [11]:
# Atribui um código sequencial quando customer_id está nulo.
df_backup = pd.DataFrame( df_missing['invoice_no'].drop_duplicates() )
df_backup['customer_id'] = np.arange( 1900, 1900 + len( df_backup ), 1 )

# Merge com o dataframe original
df1 = pd.merge( df1, df_backup, on='invoice_no', how='left')

# Coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first( df1['customer_id_y'] )

# Drop colunas temporárias
df1 = df1.drop( columns=['customer_id_x', 'customer_id_y'], axis=1)

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


In [12]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6. Mudança de DataType

In [13]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


In [14]:
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y' )

In [15]:
df1['customer_id'] = df1['customer_id'].astype( int )

In [16]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int32
dtype: object

## 1.7. Estatística Descritiva

In [17]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'])
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1. Atributos Numéricos

In [18]:
num_attributes.describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
unit_price,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0


### <font color='red'> 1.7.1.1. Atributos Numéricos - Investigar </font>

1. Quantidade negativa ( pode ser devolução? )
2. preço unitário igual a zero. ( pode ser promoção ou bonificação? )

### 1.7.2. Atributos Categóriocos

In [19]:
cat_attributes.head()

,invoice_no,stock_code,description,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom,17850


#### Invoice No

In [20]:
# identificar se tem caracter no invoice_no
df_char_invoice = df1.loc[df1['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), :]

char_invoice_total = df_char_invoice.shape[0]
negative_char_invoice = df_char_invoice[df_char_invoice['quantity'] < 0].shape[0]

print( f'Qt Negativa : {negative_char_invoice} \nCom Caracter: {char_invoice_total} ' )

Qt Negativa : 9288 
Com Caracter: 9291 


#### Stock code

In [21]:
# Identifica stock_code com apenas caracteres.
stock_only_char = cat_attributes.loc[cat_attributes['stock_code'].apply( lambda x: bool( re.search( '^[a-zA-z]+$', x ) ) ), 'stock_code'].unique()
stock_only_char

# Ação:
# Remover stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

#### Description

In [22]:
df1.head()

# Ação:
# Deletar description, pois aparentemente não tem influência no fenomeno.

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


#### Country

In [23]:
df1['country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [24]:
df1['country'].value_counts( normalize=True ).head()

United Kingdom    0.914320
Germany           0.017521
France            0.015790
EIRE              0.015124
Spain             0.004674
Name: country, dtype: float64

In [25]:
df1[['customer_id', 'country']].drop_duplicates().groupby( 'country' ).count().reset_index().sort_values( 'customer_id', ascending=False).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


# 2.0. Filtragem de Variáveis
Adicionado a filtragem de variáveis antes do Feature engeneering pois como tem valores que precisavam ser tratados, o erro seria apenas propagado.

In [26]:
df2 = df1.copy()

In [27]:
## -> Atributos numéricos
# Unit Price > 0.0
df2 = df2[df2['unit_price'] > 0.01]

# Stock code != ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
df2 = df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'] )]

# Description
df2 = df2.drop( columns='description', axis=1 )

# Map
df2 = df2[~df2['country'].isin( ['European Comunity', 'Unspicified' ] )]

# Quantidade - Números negativos
df2_returns = df2[df2['quantity'] < 0]
df2_purchases = df2[df2['quantity'] > 0]

In [28]:
df2_returns.head()

,invoice_no,stock_code,quantity,invoice_date,unit_price,country,customer_id
154,C536383,35004C,-1,2016-11-29,4.65,United Kingdom,15311
235,C536391,22556,-12,2016-11-29,1.65,United Kingdom,17548
236,C536391,21984,-24,2016-11-29,0.29,United Kingdom,17548
237,C536391,21983,-24,2016-11-29,0.29,United Kingdom,17548
238,C536391,21980,-24,2016-11-29,0.29,United Kingdom,17548


# 3.0. Feature Engineering

In [29]:
# Idéias de Features
## Média móvel - 7d, 14d, 30d
## Quantidade de compras por mês, antes do dia 15 e depois do dia 15, ou por semana.
## Média referene ao financeiro
## Basket size

In [30]:
df3 = df2.copy()

## 3.1. Criação de features

In [31]:
df_ref = df3[['customer_id']].drop_duplicates( ignore_index=True )
df_ref.shape

(5797, 1)

### 3.1.1. Gross Revenue - Faturamento

In [32]:
# Gross Revenue - Faturamento
df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']

# Monetary
df_monetary = df2_purchases.loc[ : , ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left')

C:\Users\laurivan\anaconda3\envs\dsEmProducao\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [33]:
df_ref.head()

,customer_id,gross_revenue
0,17850,5391.21
1,13047,3232.59
2,12583,6705.38
3,13748,948.25
4,15100,876.00


### 3.1.2. Recency - Dias da última compra

In [34]:
# Recência - Últimno dia com compra
df_recency = ( df2_purchases.loc[ : , ['customer_id', 'invoice_date']].groupby( 'customer_id' )
                                                                      .max()
                                                                      .reset_index() )

df_recency['recency_days'] = ( df2_purchases['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()

df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left')

### 3.1.3. Quantidade de compras efetuadas

In [35]:
# Número de vendas efetuadas
df_freq = (df2_purchases[['customer_id', 'invoice_no']].drop_duplicates()
                                                      .groupby( 'customer_id' )
                                                      .count()
                                                      .reset_index()
                                                      .rename( columns={'invoice_no': 'qtde_invoices'}))

df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left')

In [36]:
df_ref.head()

,customer_id,gross_revenue,recency_days,qtde_invoices
0,17850,5391.21,372.0,34.0
1,13047,3232.59,56.0,9.0
2,12583,6705.38,2.0,15.0
3,13748,948.25,95.0,5.0
4,15100,876.00,333.0,3.0


### 3.1.4. Quantidade de produtos comprados

In [37]:
# Quantidade de produtos comprados por cliente.
df_freq = ( df2_purchases[['customer_id', 'quantity']].groupby( 'customer_id' )
                                                     .sum()
                                                     .reset_index()
                                                     .rename( columns={'quantity':'qtde_products' } ) )

df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left')
df_ref.head()

,customer_id,gross_revenue,recency_days,qtde_invoices,qtde_products
0,17850,5391.21,372.0,34.0,1733.0
1,13047,3232.59,56.0,9.0,1390.0
2,12583,6705.38,2.0,15.0,5028.0
3,13748,948.25,95.0,5.0,439.0
4,15100,876.00,333.0,3.0,80.0


### 3.1.5. Averange Ticket Value - Ticket Médio

In [38]:
# Ticket médio
df_avg_ticket = ( df2_purchases[['customer_id', 'gross_revenue']].groupby( 'customer_id' )
                                                                 .mean()
                                                                 .reset_index()
                                                                 .rename( columns={'gross_revenue': 'avg_ticket'} ) )

df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')

### 3.1.6. Averange Recency Days - Média Intervalo de Compra

In [39]:
# Média do intervalo de compra
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values( ['customer_id', 'invoice_date'], ascending=['False', 'False'] )
# Próximo Cliente
df_aux['next_customer_id'] = df_aux['customer_id'].shift()
# Próxima Data
df_aux['previous_date'] = df_aux['invoice_date'].shift()

df_aux['avg_recency_days'] = df_aux.apply( lambda x: x['previous_date'] if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1 )

df_aux = df_aux.drop( ['invoice_date', 'next_customer_id', 'previous_date'], axis=1 ).dropna()

# average recency 
df_avg_recency_days = df_aux.groupby( 'customer_id' ).mean().reset_index()

# merge
df_ref = pd.merge( df_ref, df_avg_recency_days, on='customer_id', how='left' )

df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtde_invoices         91
qtde_products         91
avg_ticket            91
avg_recency_days    2822
dtype: int64

### 3.1.7. Frequência de Compra

In [40]:
df_aux = ( df2[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                             .groupby( 'customer_id')
                                                             .agg( max_ = ( 'invoice_date', 'max' ), 
                                                                   min_ = ( 'invoice_date', 'min' ),
                                                                   days_= ( 'invoice_date', lambda x: ( ( x.max() - x.min() ).days ) + 1 ),
                                                                   buy_ = ( 'invoice_no', 'count' ) ) ).reset_index()

# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if  x['days_'] != 0 else 0, axis=1 )

# Merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left' )

df_ref.isna().sum()    

customer_id            0
gross_revenue         91
recency_days          91
qtde_invoices         91
qtde_products         91
avg_ticket            91
avg_recency_days    2822
frequency              0
dtype: int64

### 3.1.8. Devoluções

In [41]:
# # Número de devoluções
# df_returns = df2_returns[['customer_id', 'invoice_no']].groupby( 'customer_id' ).count().reset_index().rename( columns={'invoice_no': 'returns'})
# df_ref = pd.merge( df_ref, df_returns, on='customer_id', how='left')
# df_ref.loc[df_ref['returns'].isna(), 'returns'] = 0

In [42]:
# Soma quantidade de devoluções
df_returns = ( df2_returns[['customer_id', 'quantity']].groupby( 'customer_id' )
                                                       .sum()
                                                       .reset_index()
                                                       .rename( columns={'quantity': 'qtde_returns'} ) )

df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, on='customer_id', how='left')
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()  

customer_id            0
gross_revenue         91
recency_days          91
qtde_invoices         91
qtde_products         91
avg_ticket            91
avg_recency_days    2822
frequency              0
qtde_returns           0
dtype: int64

### 3.1.9. Basket Size - Quantidade de Itens por Cesta - Quantity
- Invoice no = compra = purchase
- stock code = Produto = Product
- Quantity = Item = Item

In [43]:
df_aux = ( df2_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby( 'customer_id' )
                                                                          .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                n_products=( 'quantity', 'sum' ) )
                                                                          .reset_index() )

# Cálculo média
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id' )
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtde_invoices         91
qtde_products         91
avg_ticket            91
avg_recency_days    2822
frequency              0
qtde_returns           0
avg_basket_size       91
dtype: int64

### 3.1.10. Unique Basket Size - Quantidade de produtos distintos por compra
- Ver se realmente é a melhor forma de calcular esta feature e qual o retorno dela

In [44]:
 df_aux = ( df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id' )
                                                                             .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                    n_products=( 'stock_code', 'nunique' ) )
                                                                             .reset_index() )

# calculation
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id' )

df_ref.isna().sum()

customer_id                  0
gross_revenue               91
recency_days                91
qtde_invoices               91
qtde_products               91
avg_ticket                  91
avg_recency_days          2822
frequency                    0
qtde_returns                 0
avg_basket_size             91
avg_unique_basket_size      91
dtype: int64

In [45]:
df_ref.isna().sum()

customer_id                  0
gross_revenue               91
recency_days                91
qtde_invoices               91
qtde_products               91
avg_ticket                  91
avg_recency_days          2822
frequency                    0
qtde_returns                 0
avg_basket_size             91
avg_unique_basket_size      91
dtype: int64

In [46]:
# df_ref = df3[['customer_id']].drop_duplicates( ignore_index=True )
# # Gross Revenue - Faturamento
# df3['gross_revenue'] = df3['quantity'] * df3['unit_price']

# # Monetary
# df_monetary = df3[['customer_id', 'gross_revenue']].groupby( 'customer_id' ).sum().reset_index()
# df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left')

# # Recência - Últimno dia com compra
# df_recency = df3[['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
# df_recency['recency_days'] = ( df3['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
# df_recency = df_recency[['customer_id', 'recency_days']].copy()
# df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left')

# # Frequência
# df_freq = df3[['customer_id', 'invoice_no']].drop_duplicates().groupby( 'customer_id' ).count().reset_index()
# df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left')

# # Ticket médio
# df_avg_ticket = df3[['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue': 'avg_ticket'})
# df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')

# 4.0. Análise Exploratória dos Dados - EDA

In [47]:
df4 = df_ref.dropna().copy()

## Sweetviz

In [48]:
import sweetviz as sv

my_report = sv.analyze(df4)
my_report.show_html()

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## Pandas Profiling

In [49]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport( df4 )
# profile.to_html("your_report.html")
# profile.to_notebook_iframe()

## 4.1. Análise Univariada

### 4.1.1. Gross Revenue

In [50]:
 df4.sort_values( 'gross_revenue', ascending=False ).head()

,customer_id,gross_revenue,recency_days,qtde_invoices,qtde_products,avg_ticket,avg_recency_days,frequency,qtde_returns,avg_basket_size,avg_unique_basket_size
1017,14646,279138.02,1.0,72.0,196844.0,135.503893,2017-06-26 11:12:00.000000000,0.206215,288.0,2733.944444,9.708333
492,18102,259657.30,0.0,60.0,64124.0,602.453132,2017-07-04 08:38:24.000000000,0.163043,0.0,1068.733333,2.500000
434,17450,194550.79,8.0,46.0,69993.0,577.302047,2017-07-18 22:17:08.571428608,0.136111,952.0,1521.586957,2.695652
3341,16446,168472.50,0.0,2.0,80997.0,56157.500000,2017-05-16 00:00:00.000000000,0.014563,80995.0,40498.500000,1.500000
66,14911,140450.72,1.0,199.0,80263.0,24.757751,2017-07-03 16:16:46.993007104,0.651475,3332.0,403.331658,8.974874


In [51]:
 df4[df4['customer_id'] == 14646].head()

,customer_id,gross_revenue,recency_days,qtde_invoices,qtde_products,avg_ticket,avg_recency_days,frequency,qtde_returns,avg_basket_size,avg_unique_basket_size
1017,14646,279138.02,1.0,72.0,196844.0,135.503893,2017-06-26 11:12:00,0.206215,288.0,2733.944444,9.708333


### 4.1.2. Qtde Items

### 4.1.3. Qtde Products

## 4.2. Análise Bivariada

## 4.2. Análise Multivariada

# 5.0. Preparação dos Dados

In [52]:
df5 = df4.copy()

## 5.1. Scaler

In [53]:
ss = pp.StandardScaler()

df5['gross_revenue'] = ss.fit_transform( df5[['gross_revenue']] )
df5['recency_days'] = ss.fit_transform( df5[['recency_days']] )
df5['invoice_no'] = ss.fit_transform( df5[['invoice_no']] )
df5['avg_ticket'] = ss.fit_transform( df5[['avg_ticket']] )
df5['returns'] = ss.fit_transform( df5[['returns']] )

KeyError: "None of [Index(['invoice_no'], dtype='object')] are in the [columns]"

In [ ]:
df5.head()

# 6.0. Seleção de Features

In [ ]:
df6 = df5.copy()

# 7.0. Hyperparameter Fine-Tunning

In [ ]:
X = df6.drop( columns=['customer_id'] )

In [ ]:
clusters = [2, 3, 4, 5, 6, 7]

## 7.1. Within-Cluster Sum of Square (WSS)

In [ ]:
wss = []

for k in clusters:
    # Model Definition
    kmeans = c.KMeans( init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)
    
    # model taining
    kmeans.fit( X )
    
    # Validation
    wss.append( kmeans.inertia_ )
    
# plot wss

plt.figure(figsize=(14,7))
plt.plot( clusters, wss, linestyle='--', marker='o', color='b' )
plt.xlabel( 'K')
plt.ylabel( 'Within-Cluster Sum of Square' )
plt.title( 'WSS vs K')

## 7.2. Elbow

In [ ]:
plt.figure(figsize=(14,7))
kmeans = KElbowVisualizer( c.KMeans(), k=clusters, timings=False)
kmeans.fit( X )
kmeans.show()

## 7.3. Silhouette Score

In [ ]:
plt.figure(figsize=(14,7))
kmeans = KElbowVisualizer( c.KMeans(), k=clusters, metric='silhouette', timings=False)
kmeans.fit( X )
kmeans.show()

## 7.4. Silhouette Analysis

Implementar nos proximos ciclos https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

In [ ]:
# Para identificar qual a melhor quantidade de clusters vamos plotar a silhouette de cada um e fazer uma análise visual.

fig, ax = plt.subplots( 3, 2, figsize=(25, 18) )

for k in clusters:
    km = c.KMeans( init='random', n_clusters=k, n_init=10, max_iter=100, random_state=42)
    
    q, mod = divmod( k, 2 )
    visualizer = SilhouetteVisualizer( km, color='yellowbrick', ax=ax[q-1][mod] )
    visualizer.fit( X )
    visualizer.finalize()

# 8.0. Modelagem de Machine Learning

## 8.1. K-Means

In [ ]:
k = 3

# Model Definition
kmeans = c.KMeans( init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)

# model taining
kmeans.fit( X )

# clustering
labels = kmeans.labels_

## 8.2. Cluster Validation

In [ ]:
print( f'WSS Value: {kmeans.inertia_}' )
print( 'SS Value: {}'.format( m.silhouette_score( X, labels, metric='euclidean') ) )

# 9.0. Análise dos clusters

In [ ]:
df9 = df6.copy()
df9['cluster'] = labels

In [ ]:
df9.columns


## 9.1. Silhouette

In [ ]:
visualizer = SilhouetteVisualizer( kmeans, color='yellowbrick' )
visualizer.fit( X )
visualizer.finalize()

## 9.2. 2d Plot

In [ ]:
df9['cluster'].value_counts(normalize=True) * 100

In [ ]:
# plt.figure( figsize=(35, 18) )
df_viz = df9.drop( columns='customer_id', axis=1 )
sns.pairplot( df_viz, hue='cluster' )

## 9.3. UMAP

In [ ]:
reducer = umap.UMAP( n_neighbors=20, random_state=42 )
embedding = reducer.fit_transform( X )

# Embedding
df_viz['embedding_x'] = embedding[:, 0]
df_viz['embedding_y'] = embedding[:, 1]

# Plot UMAP
plt.figure( figsize=(11, 7) )
sns.scatterplot( x='embedding_x', 
                y='embedding_y',
               hue='cluster',
               palette=sns.color_palette( 'hls', n_colors=len( df_viz['cluster'].unique() ) ),
               data=df_viz)

## 9.4. Inspeção Visual

In [ ]:
# Esta inspeção serve para quado temos 3 dimensões
# df_tmp = df9[['recency_days', 'invoice_no', 'gross_revenue', 'cluster']]
fig = px.scatter_3d( df9, x='recency_days', y='invoice_no', z='gross_revenue', color='cluster')
fig.show()

## 9.5. Perfil do Cluster

In [ ]:
# Número de clientes
df_cluster = df9[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] =  100 * ( df_cluster['customer_id'] / df_cluster['customer_id'].sum() )

# Média Faturamento
df_avg_gross_revenue = df9[['gross_revenue', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, on='cluster', how='inner')

# Média Recencia
df_avg_recency_days = df9[['recency_days', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, on='cluster', how='inner')

# Média compras
df_avg_invoice_no = df9[['invoice_no', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_invoice_no, on='cluster', how='inner')

# Média ticket
df_avg_ticket = df9[['avg_ticket', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_ticket, on='cluster', how='inner')

In [ ]:
df_cluster.head()

In [ ]:
## df9[['customer_id', 'cluster']]['cluster'].value_counts(normalize=True) * 100

### Cluster 01: (candidato à Insider)
- Número de Clientes: 6 (0,14%)
- Recência média: 7 Dias
- Compras Média: 89 Compras
- Receita média: $ 182.182,00 Dólares

### Cluster 02:
- Número de Clientes: 31 (99%)
- Recência média: 14 Dias
- Compras Média: 53 Compras
- Receita média: $ 40.543,52 Dólares

### Cluster 03: 
- Número de Clientes: 4.335 (99%)
- Recência média: 92 Dias
- Compras Média: 5 Compras
- Receita média: $ 1.372,57 Dólares

# 10.0. Deploy Modelo em Produção